In [ ]:
from langgraph.graph import StateGraph, MessagesState, START, END
from IPython.display import Image,Markdown,display_markdown ,display
import getpass, os, subprocess
from send2trash import send2trash
from pathlib import Path
from langchain_google_genai import ChatGoogleGenerativeAI
from google.api_core import exceptions
from langchain_core.messages import HumanMessage,SystemMessage, ToolMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.tools import tool
from tavily import TavilyClient
from typing import Literal
import pygetwindow as gw
import pyautogui, time
from rapidfuzz import process, fuzz
from pycaw.pycaw import AudioUtilities
import comtypes
import screen_brightness_control as sbc

In [ ]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")
if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your TAVILY API key: ")
    

# TOOLS

In [ ]:
tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

#### Research and Web_scraper

In [ ]:
@tool
def internet_search(
    query: str,
    max_results: int = 4,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    """
    Perform an online web search using the Tavily Search API.

    USE THIS TOOL WHEN:
    - You need **current, public information, any info** from the internet.
    - You want **summaries of web pages**.
    - You are answering questions about:
        • general knowledge or trending topics
        • recent news or current events, sports, weather
        • financial information, markets, companies, or economic data
    - The user asks you to "search", "look up", "find online", or "check on the internet" or "go" or related keywords.

    DO NOT USE THIS TOOL WHEN:
    - The information is already provided in the conversation.
    - The answer is something you can generate from reasoning alone.

    Args:
    - query (str): The text query you want to search for.
    - max_results (int): Number of results to retrieve (default 4).
    - topic (str): Search domain — "general", "news", or "finance".
      Use:
        • "general" for broad informational searches.
        • "news" for recent events or breaking updates.
        • "finance" for markets, stocks, economics, or company profiles.
    - include_raw_content (bool): If True, includes raw webpage text in results.

    RETURNS:
    - A structured list of search results with titles, URLs, summaries,
      and optionally raw text.
    """
    return tavily_client.search(
        query,
        max_results=max_results if max_results is not None else 4,
        include_raw_content=include_raw_content,
        topic=topic,
    )


In [ ]:
@tool
def web_scraper(urls: list[str]):
    """
    Extract/Scrape detailed content directly from any specific webpages using Tavily's extractor.
    This tool can also scrape social media websites (LinkedIn, Instagram, Facebook, Twitter).
    USE THIS TOOL WHEN:
    - You already have one or more **URLs/Links** and need to read their content.
    - You want to access **full page text**, **HTML-derived content**, or **structured sections**.
    - The task requires:
        • reading an article directly
        • extracting facts, tables, lists, or deep page information
        • verifying claims from a specific link
        • doing analysis on a known webpage
    - The user provides URLs/link or you have a link/url and asks to "scrape", "extract", "read", or "get content from" or "go and tell from" or "get more info about" or related keywords.

    DO NOT USE THIS TOOL WHEN:
    - You do not know about which link/url — use internet_search instead.
    - The information is already provided in the conversation.

    Args:
    - urls (list[str]): One or more URLs to scrape.

    RETURNS:
    - Detailed extracted content from each provided URL.
    """
    if isinstance(urls, str):
        urls = [urls]

    return tavily_client.extract(
        urls,
        extract_depth="advanced",
        format="markdown"
    )


#### Close/minimize/maximize/restore apps

In [ ]:
def find_closest_window_title(query : str):
    titles = gw.getAllTitles()
    result = process.extractOne(query, titles, scorer=fuzz.partial_ratio)
    return result[0] if result else None

In [ ]:
@tool
def close_app(window_name: str):
    """
    Closes the desktop application window.

    Args:
        window_name (str): The title/name (full or partial) of the window
        you want to close.

    Returns:
        str: Confirmation message indicating whether the matching
        window was successfully closed or not found.
    """
    window_to_close = find_closest_window_title(window_name)
    if window_to_close:
        window_handle = gw.getWindowsWithTitle(window_to_close)[0]
        window_handle.close()
        return f"{window_to_close} was closed "
    else:
        return f"No window found with a title close to '{window_name}'"

In [ ]:
@tool
def minimize_app(window_name: str):
    """
    Minimizes the desktop window.

    Args:
        window_name (str): The full or partial title/name of the window
        you want to minimize.

    Returns:
        str: Message indicating whether the matching window was
        successfully minimized or not found.
    """
    window_to_minimize = find_closest_window_title(window_name)
    if window_to_minimize:
        window_handle = gw.getWindowsWithTitle(window_to_minimize)[0]
        window_handle.minimize()
        return f"{window_to_minimize} was minimized "
    else:
        return f"No window found with a title close to '{window_name}'"


In [ ]:
@tool
def maximize_app(window_name: str):
    """
    Maximizes the desktop window.

    Args:
        window_name (str): The full or partial title/name of the window
        you want to maximize.

    Returns:
        str: Message indicating whether the matching window was
        successfully maximized or not found.
    """
    window_to_maximize = find_closest_window_title(window_name)
    if window_to_maximize:
        window_handle = gw.getWindowsWithTitle(window_to_maximize)[0]
        window_handle.minimize()
        window_handle.maximize()
        window_handle.activate()
        return f"{window_to_maximize} was maximized "
    else:
        return f"No window found with a title close to '{window_name}'"


In [ ]:
@tool
def restore_app(window_name: str):
    """
    Restores a minimized or maximized desktop window.

    Args:
        window_name (str): The full or partial title/name of the window
        you want to restore.

    Returns:
        str: Message indicating whether the matching window was
        successfully restored or not found.
    """
    window_to_restore = find_closest_window_title(window_name)
    if window_to_restore:
        window_handle = gw.getWindowsWithTitle(window_to_restore)[0]
        window_handle.minimize()
        window_handle.restore()
        window_handle.activate()
        return f"{window_to_restore} was restored "
    else:
        return f"No window found with a title close to '{window_name}'"


#### Open apps

In [ ]:
def get_all_file_paths_and_names(directory_path):
    """
    Gets the full path and the name (without extension) of all files 
    in the specified directory and its subdirectories using pathlib.

    Returns:
        A list of dictionaries, where each dictionary contains:
        - 'path': The full absolute path string.
        - 'name': The file name without the extension.
    """
    path = Path(directory_path)
    file_data = []

    for item in path.glob('**/*'):
        if item.is_file():
            # Get the full absolute path as a string
            full_path = str(item.resolve()) 
            
            # Get the file name without the extension
            name_without_ext = item.stem.lower()
            
            file_data.append({
                'path': full_path,
                'name': name_without_ext
            })

    return file_data

directory = 'C:\\ProgramData\\Microsoft\\Windows\\Start Menu\\Programs'
directory2 = f"{os.path.expanduser('~')}\\AppData\\Roaming\\Microsoft\\Windows\\Start Menu\\Programs"

In [ ]:
@tool
def open_app(window_name: str):
    """
    Opens a Windows application by any provided name. Valid and invalid both names are allowed.

    Args:
        window_name (str): Name of any app or executable.

    Returns:
        str: Success or error message.
    """
    all_apps_data = get_all_file_paths_and_names(directory)
    extra_apps = get_all_file_paths_and_names(directory2)
    all_apps_data.extend(extra_apps)


    appname = [app['name'] for app in all_apps_data]
    app_paths = [app['path'] for app in all_apps_data]


    names_lower = [name.lower() for name in appname]

    match = process.extractOne(
        window_name.lower(),
        names_lower,
        scorer=fuzz.partial_ratio
    )

    if match is None or match[1] < 70:
        pyautogui.press("win")
        time.sleep(1)
        pyautogui.write(window_name)
        return f"{window_name} was not an application, so it was searched in the Windows search bar."

    
    matched_name_lower = match[0]

    index = names_lower.index(matched_name_lower)

    final_path = app_paths[index]
    original_name = appname[index]

    try:
        subprocess.Popen(final_path, shell=True)
        return f"'{original_name}' has been opened successfully."
    except Exception as e:
        return f"Failed to open '{original_name}': {e}"


#### Switch btwn Apps

In [ ]:
@tool
def switch_btwn_apps(window_name: str):
    """
    Switch between apps just like alt+tab.

    Args:
        window_name (str): The full or partial title/name of the window app
        you want to switch to.

    Returns:
        str: Message indicating whether the matching window was
        successfully switched or not found.
    """
    window_to_switch = find_closest_window_title(window_name)
    if window_to_switch:
        window_handle = gw.getWindowsWithTitle(window_to_switch)[0]
        if not window_handle.isMaximized:
            window_handle.minimize()
            window_handle.restore()
            window_handle.activate()
            return f"{window_to_switch} is active now. "
        else:
            window_handle.minimize()
            window_handle.maximize()
            window_handle.activate()
            return f"{window_to_switch} is active now. "
    else:
        return f"No window found with a title '{window_name}' to switch to."


### Control brightness/Volume

In [ ]:
@tool
def set_volume(action: Literal["set_to", "increase_by", "decrease_by", "current_vol"], amount: int|None ):
    """ 
    Sets the system volume to the specified percentage level. Also gets the current volume level.
        - 0 = Mute
        - 50 = Half
        - 100 = Full
    
    Args:
        amount (int): The target volume level as a percentage (0-100). Must be an integer between 0 and 100, inclusive.
            - Can be None if action is "current_vol".
        action (str): The action to perform on the volume. It can be one of the following:
            - "set_to": Set the volume to the specified amount.
            - "increase_by": Increase the current volume by the specified amount.
            - "decrease_by": Decrease the current volume by the specified amount.
            - "current_vol": Get the current volume level.
    
    Returns:
        Success message indicating the previous and new volume levels, or an Error message if the input is out of range.
    """
    
    if action == "current_vol":
        try:
            comtypes.CoInitialize()
            devices = AudioUtilities.GetSpeakers()
            volume = devices.EndpointVolume
            current_scalar = volume.GetMasterVolumeLevelScalar()
            current_percent = int(current_scalar * 100)
            return f"The current system volume is {current_percent}%."
        except Exception as e:
            return f"An error occurred while retrieving the current volume: {e}"
        finally:
            try:
                comtypes.CoUninitialize()
            except:
                pass

    if not (0 <= amount <= 100):
        return f"Enter a value between 0-100 inclusive. {amount}% is invalid."

    if action not in ["set_to", "increase_by", "decrease_by"]:
        return f"Invalid action '{action}'. Use 'set_to', 'increase_by', or 'decrease_by'."

    # Start COM session
    try:
        comtypes.CoInitialize()
    except Exception as e:
        return f"COM initialization failed: {e}"

    try:
        
        devices = AudioUtilities.GetSpeakers()
        volume = devices.EndpointVolume

        
        current_scalar = volume.GetMasterVolumeLevelScalar()
        current_percent = int(current_scalar * 100)

        
        if action == "set_to":
            new_scalar = amount / 100
            volume.SetMasterVolumeLevelScalar(new_scalar, None)
            return f"Previous volume was {current_percent}%. Volume set to {amount}% successfully."

        
        if action == "increase_by":
            new_percent = current_percent + amount
            if new_percent > 100:
                return (
                    f"Volume cannot be increased by {amount}%. "
                    f"Current: {current_percent}%. Max allowed: 100%."
                )
            new_scalar = new_percent / 100
            volume.SetMasterVolumeLevelScalar(new_scalar, None)
            return (
                f"Previous volume was {current_percent}%. "
                f"Volume increased by {amount}%. New volume: {new_percent}%."
            )


        if action == "decrease_by":
            new_percent = current_percent - amount
            if new_percent < 0:
                return (
                    f"Volume cannot be decreased by {amount}%. "
                    f"Current: {current_percent}%. Min allowed: 0%."
                )
            new_scalar = new_percent / 100
            volume.SetMasterVolumeLevelScalar(new_scalar, None)
            return (
                f"Previous volume was {current_percent}%. "
                f"Volume decreased by {amount}%. New volume: {new_percent}%."
            )

    except Exception as e:
        return f"An error occurred while controlling volume: {e}"

    finally:
        try:
            comtypes.CoUninitialize()
        except:
            pass


In [ ]:
@tool
def set_brightness(action: Literal["set_to", "increase_by", "decrease_by", "current_brt"], level: int):
    """
    Sets the system brightness to the specified percentage level. Also get the current brightness level.
        - 0 = Minimum brightness
        - 50 = Medium brightness
        - 100 = Maximum brightness

    Args:
        action (str): The action to perform on the brightness. It can be one of the following:
            - "set_to": Set the brightness to the specified level.
            - "increase_by": Increase the current brightness by the specified level.
            - "decrease_by": Decrease the current brightness by the specified level.
            - "current_brt": Get the current brightness level.
        level (int): The target brightness level as a percentage Multiple of 10 (0, 10, 20, ..., 100).

    Returns:
        str: Success message indicating the previous and new brightness levels,
             or an Error message if the input is out of range.
    """
    if action not in ["set_to", "increase_by", "decrease_by", "current_brt"]:
        return f"Invalid action '{action}'. Use 'set_to', 'increase_by', 'decrease_by', or 'current_brt'."
    if level not in range(0, 101, 10):
        return f"Enter a value between 0-100 inclusive in multiples of 10. {level}% is invalid."
    
    current_brightness = sbc.get_brightness(display=0)[0]
    
    if action == "current_brt":
        return f"The current system brightness is {current_brightness}%."
    
    elif action == "set_to":
        sbc.set_brightness(level, display=0, no_return=False)
        return f"Previous brightness was {current_brightness}%. Brightness set to {level}% successfully."
    
    elif action == "increase_by":
        sbc.set_brightness(f'+{level}', display=0)
        new_brightness = sbc.get_brightness(display=0)[0]
        return (
            f"Previous brightness was {current_brightness}%. "
            f"Brightness increased by {level}%. New brightness: {new_brightness}%."
        )
        
    elif action == "decrease_by":
        sbc.set_brightness(f'-{level}', display=0)
        new_brightness = sbc.get_brightness(display=0)[0]
        return (
            f"Previous brightness was {current_brightness}%. "
            f"Brightness increased by {level}%. New brightness: {new_brightness}%."
        )


#### Create/Rename/Delete Folder

In [ ]:
@tool
def create_folder(name_of_folder: str):
    """
    Creates a new folder or folder in folder at Desktop directory with the specified name of folder.

    Args:
        name_of_folder (str): The name of folder or folder in folder to be created. It can include subfolder names separated by forward slashes (e.g., "NewFolder/xyz/yz/as/pi").

    Returns:
        str: Success or error message.
    """
    home_dir = os.path.expanduser("~")
    desktop_dir = os.path.join(home_dir, "Desktop")
    folder_path = os.path.join(desktop_dir, name_of_folder)
    normalized_path = os.path.normpath(folder_path)
    try:
        os.makedirs(normalized_path, exist_ok=False)
        return f"Folder created successfully at: {normalized_path}"
    except FileExistsError:
        return f"Error: A folder already exists at: {normalized_path}"
    except Exception as e:
        return f"Error creating folder at {normalized_path}: {e}"

In [ ]:
@tool
def rename_folder(current_name: str, new_name: str):
    """
    Renames an existing folder on the Desktop after asking user for confirmation.

    Args:
        current_name (str): The current name of the folder to be renamed. It can include subfolder names separated by forward slashes (e.g., "OldFolder/xyz").
        new_name (str): The new name for the folder. It can include subfolder names separated by forward slashes (e.g., "NewFolder/abc").

    Returns:
        str: Success or error message.
    """
    home_dir = os.path.expanduser("~")
    desktop_dir = os.path.join(home_dir, "Desktop")
    current_path = os.path.normpath(os.path.join(desktop_dir, current_name))
    new_path = os.path.normpath(os.path.join(desktop_dir, new_name))
    try:
        os.rename(current_path, new_path)
        return f"Folder renamed successfully from {current_path} to {new_path}"
    except FileNotFoundError:
        return f"Error: The folder '{current_path}' does not exist."
    except FileExistsError:
        return f"Error: A folder with the name '{new_path}' already exists."
    except Exception as e:
        return f"Error renaming folder: {e}"

In [ ]:
@tool
def delete_folder(name_of_folder: str):
    """
    Removes and send an existing folder or folder in folder from Desktop directory to Recycle Bin with the specified name of folder after asking for confirmation. [Do you want to delete <folder_name>?]

    Args:
        name_of_folder (str): The name of folder or folder in folder to be deleted. It can include subfolder names separated by forward slashes (e.g., "NewFolder/xyz/yz/as/pi").

    Returns:
        str: Success or error message.
    """
    home_dir = os.path.expanduser("~")
    desktop_dir = os.path.join(home_dir, "Desktop")
    folder_path = os.path.join(desktop_dir, name_of_folder)
    normalized_path = os.path.normpath(folder_path)
    try:
        send2trash(normalized_path)
        return f"Folder deleted successfully at: {normalized_path}"
    except FileNotFoundError:
        return f"Error: No folder found at: {normalized_path}"
    except OSError as e:
        return f"Error deleting folder at {normalized_path}: {e}"

#### Create/Rename/Delete File

In [ ]:
@tool
def create_add_content_file(file_name: str, content: str = ""):
    """
    Creates a new file on the Desktop and writes optional content. Ask to enter file extension if not provided in file_name. Also ask if something to write in the file. This tool can also be used to add content in existing file.
    
    Args:
        file_name (str): The name of the file to be created. It should include the file extension (e.g., "example.txt").
        content (str, optional): The content to write into the file. Defaults to an empty string.
        
    Returns:
        str: Success or error message.
    """

    home_dir = os.path.expanduser("~")
    desktop_dir = os.path.join(home_dir, "Desktop")
    file_path = os.path.join(desktop_dir, file_name)
    normalized_path = os.path.normpath(file_path)

    try:
        with open(normalized_path, "w", encoding="utf-8") as f:
            f.write(content)

        return f"File created successfully at: {normalized_path}"
    except Exception as e:
        return f"Error creating file: {e}"


In [ ]:
@tool
def rename_file(old_name: str, new_name: str):
    """
    Renames a file after asking user for confirmation. Asks to enter file extensions if not provided in old_name or new_name.
    Args:
        old_name (str): The current name of the file to be renamed. It should include the file extension (e.g., "old_example.txt").
        new_name (str): The new name for the file. It should include the file extension (e.g., "new_example.txt").
    Returns:
        str: Success or error message.
    """

    home_dir = os.path.expanduser("~")
    desktop = os.path.join(home_dir, "Desktop")

    old_path = os.path.normpath(os.path.join(desktop, old_name))
    new_path = os.path.normpath(os.path.join(desktop, new_name))

    try:
        os.rename(old_path, new_path)
        return f"Renamed successfully:\nFROM: {old_path}\nTO:   {new_path}"
    except Exception as e:
        return f"Error renaming: {e}"


In [ ]:
@tool
def delete_file(file_name: str):
    """
    Moves a file to Recycle Bin after confirmation (Safe Delete).
    
    Args:
        file_name (str): The name of the file to be deleted.
    Returns:
        str: Success or error message.
    """

    home_dir = os.path.expanduser("~")
    desktop = os.path.join(home_dir, "Desktop")
    file_path = os.path.normpath(os.path.join(desktop, file_name))

    try:
        send2trash(file_path)
        return f"File moved to Recycle Bin: {file_path}"
    except Exception as e:
        return f"Error deleting file: {e}"

#### Move Folder

# LLM

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    
)
tools = [internet_search, web_scraper, open_app, close_app, minimize_app, maximize_app, restore_app, switch_btwn_apps, set_volume, set_brightness, create_folder, rename_folder, delete_folder, create_add_content_file, rename_file, delete_file]
tools_by_name = {tool.name: tool for tool in tools}
llmwithtools = llm.bind_tools(tools)

# NODE

In [ ]:
system_msg = SystemMessage(content=
    "You are Neura_Command, an Agentic AI created by Abhinav Ranjan Jha. "
    "You can control the entire computer system through provided tools. "
    "USE internet_search to search the internet for recent information. "
    "USE web_scraper to know all about given link/url or extract/scrape information from it. All types of websites are supported "
    "If the user provides an invalid Windows application name that you don't know, treat it as valid and proceed. "
    "More tools will be added later. Use tools when needed.\n\n"
    "After using internet_search or web_scraper, list the final URLs under a 'Sources:' section at last "
    "for ex:\n"
    "Sources:\n"
    "1. www.example.com\n"
    "2. www.example2.com\n"
    "etc.\n"
    "Always respond in Markdown, stay accurate, logical, and agentic and always do what user says no excuses."
)


In [ ]:
def call_llm_node(state: MessagesState):
    
    messages = state["messages"]
    if not any(msg.__class__.__name__ == "SystemMessage" for msg in messages):
        messages.insert(0, system_msg)
        
    print("\n" + "-----"*10 + " All Messages content " + "-----"*10)
    for msg in messages:
        if isinstance(msg, SystemMessage):
            print("system msg =", msg.content)
        elif isinstance(msg, HumanMessage):
            print("human msg =", msg.content)
        # elif isinstance(msg, ToolMessage):
        #     print(f"tool msg = {msg.content}")
        elif isinstance(msg, AIMessage):
            print("assistant msg =", msg.content)
            print("usage_metadata = ", msg.usage_metadata)
    print("-----"*10 + "End of All Messages content " + "-----"*10 + "\n")

    # print("-----"*50)
    # print(f"\nAll Messages = {"messages"}\n")
    print("-----"*10 + " Last Message content " + "-----"*10)
    print(f"\nLast message = {messages[-1]}\n")
    print("-----"*10 + "End of Last Message content " + "-----"*10)
    response = llmwithtools.invoke(messages)
    return {"messages": [response]}

In [ ]:
def execute_tool_calls_node(state: MessagesState):
    """
    Executes all tool calls from the last message dynamically.

    Args:
        state (MessagesState): The current state containing messages and tool calls.

    Returns:
        dict: A dictionary with 'messages' containing the outputs of all executed tools.
    """
    last_message = state["messages"][-1]
    tool_outputs = []

    print("\n" + "-----" * 10 + " Tool Calls to be executed " + "-----" * 10)

    # Map your tool names to the actual callable functions
    tools_map = {
        "internet_search": internet_search,
        "web_scraper": web_scraper,
        "close_app": close_app,
        "minimize_app": minimize_app,
        "maximize_app": maximize_app,
        "restore_app": restore_app,
        "open_app": open_app,
        "switch_btwn_apps": switch_btwn_apps,
        "set_volume": set_volume,
        "set_brightness": set_brightness,
        "create_folder": create_folder,
        "rename_folder": rename_folder,
        "delete_folder": delete_folder,
        "create_add_content_file": create_add_content_file,
        "rename_file": rename_file,
        "delete_file": delete_file,
        # Add more tools here as needed
    }

    for tool_call in last_message.tool_calls:
        tool_name = tool_call["name"]
        args = tool_call["args"]

        if tool_name in tools_map:
            print(f"Invoking {tool_name} with args:", args)
            try:
                result = tools_map[tool_name].invoke(args)
                tool_outputs.append(ToolMessage(tool_call_id=tool_call['id'], content=str(result)))
            except Exception as e:
                tool_outputs.append(
                    ToolMessage(tool_call_id=tool_call['id'], content=f"Error executing {tool_name}: {e}")
                )
        else:
            print(f"No tool found with name '{tool_name}'")
            tool_outputs.append(
                ToolMessage(tool_call_id=tool_call['id'], content=f"No tool found with name '{tool_name}'")
            )

    print("-----" * 10 + " End of Tool Calls execution " + "-----" * 10 + "\n")

    return {"messages": tool_outputs}


In [ ]:
def should_call_tools(state: MessagesState):
    last_message = state["messages"][-1]
    if isinstance(last_message, AIMessage) and last_message.tool_calls:
        return "tools"
    else:
        return "end"

# GRAPH

In [ ]:
graph = StateGraph(MessagesState)
graph.add_node("llm_node", call_llm_node)
graph.add_node("execute_tool_calls_node", execute_tool_calls_node)

graph.add_edge(START,"llm_node")
graph.add_conditional_edges(
    "llm_node", 
    should_call_tools,
    {
        "tools": "execute_tool_calls_node",
        "end": END,
    }
)
graph.add_edge("execute_tool_calls_node", "llm_node")


checkpointer = MemorySaver()
app = graph.compile(checkpointer=checkpointer)
config = {"configurable": {"thread_id": "ARJ"}}

In [ ]:
# use the compiled app (CompiledStateGraph) which exposes the graph
# display(Image(app.get_graph().draw_mermaid_png()))

# RESULTS

In [ ]:
input1 = {"messages": [HumanMessage(content=r"create a file name hi.txt in RenamedFolder")]}

In [ ]:
solution = """
Try one of the following Solution:
    i) Wait for a minute
    ii) Restart the App
    iii) Change the model
"""
try:
    async for event in app.astream_events(input1, config):
        if event["event"] == "on_chat_model_stream":
            chunk = event["data"]["chunk"].content
            if chunk:
                print(chunk, end="", flush=True)
                
except exceptions.InvalidArgument as e:
    print("Invalid input:", e)

except exceptions.PermissionDenied as e:
    print("Permission denied:", e)

except exceptions.ResourceExhausted as e:
    # print("Rate limit exceeded:", e)
    print(solution)

except exceptions.NotFound as e:
    print("Model not found:", e)

except exceptions.InternalServerError as e:
    print("Server error:", e)

except exceptions.ServiceUnavailable as e:
    print("Service unavailable:", e)

except Exception as e:
    print("Unknown error:", e)

# CHECKPOINT TESTING 
### To view last AI message

In [ ]:
latest_checkpoint = checkpointer.get(config)
messages = latest_checkpoint["channel_values"]["messages"]
# print(messages)
last_ai_msg = None
for msg in reversed(messages):
    if msg.__class__.__name__ == "AIMessage":
        last_ai_msg = msg.content
        break
print("*"*100)
print(last_ai_msg)
print(messages[0].__class__.__name__)

In [ ]:
display_markdown(Markdown(last_ai_msg))